# Hebex_Graphene.py — Raman Map Documentation

Detailed reference for the script located at `Raman Maps and Hebex Graphene/Hebex_Graphene.py`.
This notebook-style README focuses on the mathematics, calculations, and figures produced.

## Purpose

- Load a Raman map exported in a WITec-like TSV format.
- Compute per-pixel metrics for graphene bands D, G, and 2D.
- Derive intensity ratios and the full width at half maximum (FWHM) of the 2D band.
- Generate publication-ready figures summarizing the spectral characteristics.

## Command-line interface

```text
python Hebex_Graphene.py <filename> <area_size_x> <area_size_y> [options]
```

| Argument | Default | Description |
| --- | --- | --- |
| `filename` | sample path | TSV Raman map with MultiIndex header and `(m/n)` columns |
| `area_size_x`, `area_size_y` | 50.0 | Informational area size in µm |
| `--laser` | 633.0 | Laser wavelength (nm), informational |
| `--mono_th` | 1.5 | Threshold on I(2D)/I(G) (monolayer-like) |
| `--damage_th` | 0.3 | Threshold on I(D)/I(G) (damage indicator) |
| `--min_intensity` | 0.0 | Ignore peaks below this intensity |
| `--invert_y` | `True` | Flip map vertically to match visualization |
| `--outdir` | `.` | Output directory for figures |
| `--show` | `False` | Display figures interactively |

## Input format assumptions

- TSV (tab-separated) with a two-level header (pandas MultiIndex).
- First column: wavenumbers in cm⁻¹.
- Remaining columns: spectral intensities labelled with positions like `Intensity (m/n)`.
- Positions `m` and `n` are parsed to reconstruct a cube `intensities[m, n, k]`.
- `--invert_y` flips the cube vertically when enabled (default).

## Spectral windows

- D band: [1300, 1350] cm⁻¹
- G band: [1580, 1600] cm⁻¹
- 2D band: [2600, 2700] cm⁻¹

## Peak metrics



For a band with samples `(x_i, y_i)`:



1. **Local baseline correction**

   

   $$ y_i := y_i - \min_j(y_j). $$



2. **Peak intensity and position**

   

   $$ I = \max_i(y_i), \quad x_{\text{peak}} = x_{\operatorname{argmax}(y_i)}. $$



3. **Band area (trapezoidal rule)**

   

   $$ \text{Area} = \int y\,dx \approx \sum_i \tfrac{y_i + y_{i+1}}{2}(x_{i+1} - x_i). $$



If `I < min_intensity` or data are non-finite, the metric returns NaN.


## Full width at half maximum (FWHM)



1. Compute the half-maximum level:

   

   $$ y_{1/2} = \tfrac{1}{2} I. $$



2. Locate left/right crossings using linear interpolation between neighbors $(x_1,y_1)$ and $(x_2,y_2)$:

   

   $$ x_{\text{cross}} = x_1 + \frac{y_{1/2} - y_1}{y_2 - y_1}(x_2 - x_1). $$



3. The FWHM is the distance between the two crossings:

   

   $$ \mathrm{FWHM} = x_R - x_L. $$



If crossings cannot be bracketed, FWHM is reported as NaN.


## Intensity ratios



Ratios use a small stabilizer $\varepsilon = 10^{-12}$ to avoid division by zero when $I(G) \approx 0$:



$$ R_{2D/G} = \frac{I(2D)}{I(G) + \varepsilon}, \quad R_{D/G} = \frac{I(D)}{I(G) + \varepsilon}. $$



Percentages displayed on the hexbin plot:

- **Monolayer %**: fraction of pixels with $R_{2D/G} \ge \texttt{mono\_th}$.

- **Damage %**: fraction of pixels with $R_{D/G} > \texttt{damage\_th}$.


## Figures generated

1. **Hexbin correlation** (`hexbin_intensity_correlation.png`)
   - Bivariate density of $R_{2D/G}$ vs $R_{D/G}$ with logarithmic counts.
   - Threshold lines at `mono_th` and `damage_th`.
   - Quadrant labels and percentage annotations (Monolayer, Damage).

2. **FWHM(2D) histogram** (`fwhm2d_distribution.png`)
   - Histogram with robust bin selection (Freedman–Diaconis, with fallback).
   - Vertical lines: quality threshold 32 cm⁻¹ (dashed) and median (solid).
   - Annotation box showing pixels, percentage ≤ threshold, and median.

3. **Quadrant representative spectra** (`quadrant_representative_spectra.png`)
   - 2 × 2 grid of median spectra per quadrant.
   - Shaded regions for D, G, and 2D bands.
   - Text box with pixel count and median ratios.

4. **Composite summary** (`summary_composite.png`)
   - Single-row collage embedding the above panels.

## Output summary

All figures are saved to the directory provided via `--outdir` (created if missing):
- `hexbin_intensity_correlation.png`
- `fwhm2d_distribution.png`
- `quadrant_representative_spectra.png`
- `summary_composite.png`

## Implementation notes

- May take noticeable time for very large maps because of nested loops over pixels.
- Adjust spectral windows if band positions shift significantly.
- Baseline subtraction is local to each band; additional preprocessing (e.g., background removal) can improve accuracy.
- `--invert_y` defaults to `True`; disable by modifying argparse (currently no CLI flag to turn it off).